#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter DV360 Bulk Editor Parameters
Allows bulk editing DV360 through Sheets and BigQuery.
 1. Select <b>Load Partners</b>, <b>save</b> the recipe, then run. A Sheet called DV_Editor_ will be created.
 1. In the 'Partners' sheet tab, fill in <i>Partners</i> tab filter.
 1. Select <b>Load Advertisers</b>, <b>save</b> the recipe, then run.
 1. In the 'Advertisers' sheet tab, fill in <i>Advertisers</i> tab filter.
 1. Select <b>Load Campaigns</b>, <b>save</b> the recipe, then run. Filtering by campaigns is optional.
 1. Now, select <b>Load Insertion Orders and Line Items</b>, <b>save</b> the recipe, then run.
 1. To patch, fill in changes on all tabs with colored fields.
 1. Select <i>Preview</i>, <b>save</b> the recipe, then run the recipe.
 1. Check the <b>Audit</b> and <b>Preview</b> tabs.
 1. To Patch, select <i>Patch</i>, <b>save</b> the recipe, then run.
 1. Check the <b>Success</b> and <b>Error</b> tabs.
 1. Patch can be run multiple times.
 1. Patch ONLY changes edited fields.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_dv': 'user',  # Credentials used for dv.
  'auth_sheet': 'user',  # Credentials used for sheet.
  'auth_bigquery': 'service',  # Credentials used for bigquery.
  'recipe_name': '',  # Name of Google Sheet to create.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'command': 'Load Partners',  # Action to take.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute DV360 Bulk Editor
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      '__comment__': 'Ensure dataset exists.',
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','prefix': 'DV_Editor_','kind': 'string','order': 2,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
    }
  },
  {
    'drive': {
      '__comment__': 'Copy the default template to sheet with the recipe name',
      'auth': 'user',
      'copy': {
        'source': 'https://docs.google.com/spreadsheets/d/18G6cGo4j5SsY08H8P53R22D_Pm6m-zkE6APd3EDLf2c/',
        'destination': {'field': {'name': 'recipe_name','prefix': 'DV Editor ','kind': 'string','order': 3,'default': '','description': 'Name of Google Sheet to create.'}}
      }
    }
  },
  {
    'dv_editor': {
      '__comment': 'Depending on users choice, execute a different part of the solution.',
      'auth_dv': {'field': {'name': 'auth_dv','kind': 'authentication','order': 1,'default': 'user','description': 'Credentials used for dv.'}},
      'auth_sheets': {'field': {'name': 'auth_sheet','kind': 'authentication','order': 2,'default': 'user','description': 'Credentials used for sheet.'}},
      'auth_bigquery': {'field': {'name': 'auth_bigquery','kind': 'authentication','order': 3,'default': 'service','description': 'Credentials used for bigquery.'}},
      'sheet': {'field': {'name': 'recipe_name','prefix': 'DV Editor ','kind': 'string','order': 4,'default': '','description': 'Name of Google Sheet to create.'}},
      'dataset': {'field': {'name': 'recipe_slug','prefix': 'DV_Editor_','kind': 'string','order': 5,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
      'command': {'field': {'name': 'command','kind': 'choice','choices': ['Clear Partners','Clear Advertisers','Clear Campaigns','Clear Insertion Orders and Line Items','Clear Preview','Clear Patch','Load Partners','Load Advertisers','Load Campaigns','Load Insertion Orders and Line Items','Preview','Patch'],'order': 6,'default': 'Load Partners','description': 'Action to take.'}}
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
